In [12]:
!pip install langchain_community pypdf sentence-transformers torch chromadb

In [9]:
# Ollama 모델 로드 및 테스트
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="llama3.1", temperature=0)

response = model.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")

response

AIMessage(content='겨울철에 내한성이 강한 나무로는 소나무, 느티나무, 은행나무 등이 있습니다.', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-09T04:31:07.2252252Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3498437200, 'load_duration': 22510700, 'prompt_eval_count': 27, 'prompt_eval_duration': 355766000, 'eval_count': 27, 'eval_duration': 3119361000}, id='run-f5dd9c92-97c4-4b8c-a7f4-36b343e035af-0')

In [10]:
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("경남교육청_수목관리_업무매뉴얼.pdf", extract_images=False)
pages = loader.load()
pages[0].page_content

'경남교육 2022 - 250\n수\n목 관리 업무매뉴얼\n수목 관리 \n업무매뉴얼 쾌적한 학교 공간 재구성을 위한\n 쾌적한 학교 공간 재구성을 위한\n수목 관리\n업무매뉴얼҃թҮਭ'

In [15]:
# 문장 임베딩 및 벡터 저장소 생성
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

docs = text_splitter.split_documents(pages)

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='labhnextits/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)


OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "c:\Users\kenuh\miniconda3\envs\jupy\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [14]:
# 검색 쿼리
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

docs = retriever.get_relevant_documents(query)
print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

NameError: name 'vectorstore' is not defined

In [9]:
print(docs[1].page_content)
print(docs[1].metadata)

(1) 내한성은 동절기 중 극히 낮은 온도에 견디는 능력이다. 
(가) 내한성이 강한 수종은 한대림에서 자라는 수종으로 자작나무, 오리나무, 사
시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당된다. 
(나) 내한성이 약한 수종은 삼나무, 편백, 금송, 히말라야시다, 배롱나무, 파라칸
타, 동백나무, 후박나무, 먼나무 등 주로 남부 지역에서 자라는 수종과 자목련, 사철나무, 가이즈까향나무, 능소화, 벽오동, 오동나무 등이다. 
(다) 내한성이 약한 수종은 수간을 볏짚이나 새끼 끈으로 싸 주고, 상열을 막기
{'page': 110, 'source': '경남교육청_수목관리_업무매뉴얼.pdf'}


In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)


'겨울철에 내한성이 강한 나무에는 자작나무, 오리나무, 사시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당됩니다.'

In [11]:
query = "겨울철에 추위에 약한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)

'겨울철에 추위에 약한 나무에는 잎이 없어 수형을 보기 쉽다.'